# Heatmap LST Wilayah Lembang
Visualisasi spasial suhu permukaan tanah (LST) berdasarkan data rata-rata per titik dalam batas wilayah Lembang.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.path as mpath


## 1. Load Data

In [ ]:

# Load data utama dan batas wilayah
df = pd.read_csv("Lembang_Merged_SPI_CLEANED.csv")
boundary_df = pd.read_csv("Koordinat_Polygon_Lembang.csv")


## 2. Hitung Rata-rata LST dan Filter Suhu Valid

In [ ]:

df["location"] = df["lat"].round(4).astype(str) + ", " + df["lon"].round(4).astype(str)
avg_lst = df.groupby(["lat", "lon"])["LST"].mean().reset_index()
avg_lst = avg_lst[(avg_lst["LST"] >= 5) & (avg_lst["LST"] <= 50)]


## 3. Interpolasi Grid dan Masking ke Batas Wilayah

In [ ]:

# Interpolasi grid
grid_lon = np.linspace(avg_lst["lon"].min(), avg_lst["lon"].max(), 300)
grid_lat = np.linspace(avg_lst["lat"].min(), avg_lst["lat"].max(), 300)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_z = griddata((avg_lst["lon"], avg_lst["lat"]), avg_lst["LST"], (grid_lon, grid_lat), method='linear')

# Masking polygon
boundary_coords = boundary_df[['lon', 'lat']].values
polygon_path = mpath.Path(boundary_coords)
points = np.vstack((grid_lon.flatten(), grid_lat.flatten())).T
mask = polygon_path.contains_points(points).reshape(grid_lon.shape)
masked_grid_z = np.where(mask, grid_z, np.nan)


## 4. Plot Heatmap LST dengan Grid dan Batas Wilayah

In [ ]:

plt.figure(figsize=(10, 10))
plt.contourf(grid_lon, grid_lat, masked_grid_z, levels=100, cmap="plasma", alpha=0.9)
cbar = plt.colorbar(label="LST (°C)")
plt.contour(grid_lon, grid_lat, masked_grid_z, levels=15, linewidths=0.4, colors='black', alpha=0.3)
plt.plot(boundary_coords[:, 0], boundary_coords[:, 1], color="white", linewidth=1.5)
plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4)
plt.xlabel("Longitude", fontsize=11)
plt.ylabel("Latitude", fontsize=11)
plt.title("Heatmap LST (5°C–50°C) dengan Grid Wilayah Lembang", fontsize=13)
plt.gca().set_facecolor("white")
plt.axis("equal")
plt.tight_layout()
plt.show()
